In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pymysql
from concurrent.futures import ThreadPoolExecutor
import os

def extract_and_save2(block_of_ids, iteration, query, connection):
    try:
        # Crear un cursor
        cursor = connection.cursor()
        
        # Ejecutar el query con el offset actual y el tamaño del lote
        query_with_offset = query.format(block_of_ids)
        cursor.execute(query_with_offset)
        #print(query_with_offset)
        # Obtener los resultados
        results = cursor.fetchall()
        
        # Verificar si hay resultados
        if results:
            # Crear un DataFrame a partir de los resultados
            columns = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(results, columns=columns)
            df['body'] = df['body'].replace("\n", "").replace("\r", "")
            # Guardar el DataFrame en un archivo CSV
            csv_file_path = f'extractions/dcb/abril_2024_4_{ iteration }_data.parquet'
            #df.to_csv(csv_file_path, index=False)
            # Convert DataFrame to PyArrow Table
            table = pa.Table.from_pandas(df)
            pq.write_table(table, csv_file_path)
            


            print(f'Lote { iteration } guardado en {csv_file_path}')
        else:
            print("no archivo")    
        
    except pymysql.Error as e:
        print(f'Error en la ejecución del query: {e}')
    
    finally:
        # Cerrar el cursor
        cursor.close()

def parallel_extraction(id_batches, query, max_connections, connection):
    with ThreadPoolExecutor(max_workers=max_connections) as executor:
        for batch in id_batches:
            executor.submit(extract_and_save2, batch, query, connection)

def getAlreadyProcessedLoanIds():
    # Directorio que contiene los archivos CSV
    directorio = '/Users/sesaicornejorodriguez/baubap/demos/extractions/dcb/'

    # Lista para almacenar los DataFrames de los archivos CSV
    ids_unicos = set()

    # Enumerar todos los archivos en el directorio
    for archivo in os.listdir(directorio):
        if archivo.endswith('.parquet'):
            # Construir la ruta completa del archivo
            ruta_archivo = os.path.join(directorio, archivo)
            
            # Leer solo la columna 'loanId' del archivo CSV y agregar los IDs únicos al conjunto
            df = pd.read_csv(ruta_archivo, usecols=['loanId'])
            ids_unicos.update(df['loanId'])

    ids_concatenados = ','.join(str(id) for id in ids_unicos)

    return ids_concatenados


# Configuración de la conexión a la base de datos
db_config = {
    'host': 'baubap-prod-aurora3-mysql8-cluster.cluster-ro-cdtaivnuur7p.us-east-2.rds.amazonaws.com',
    'user': '',
    'password': '',
    'database': 'baubap',
}

# Establecer conexión a la base de datos
connection = pymysql.connect(**db_config)

try:

    # Variables de configuración
    limit = 1000
    block_size = 5000
    max_connections = 1

    # Query de extracción (debes proporcionar tu propio query)
    extract_query = """WITH L AS (
            select loanId from loans L
            where timestamp between UNIX_TIMESTAMP('2024-04-01') and UNIX_TIMESTAMP('2024-04-05')
            and L.loanIdx = 0
            and status not in ('standby')
            and loanId in ({})
        )
        SELECT S.smsLogId ,S.loanId, S.userId, S.body
        FROM smsLog S
        JOIN L ON S.loanId = L.loanId;"""

    #alreadyProcessedLoanIds = getAlreadyProcessedLoanIds()
    #print(alreadyProcessedLoanIds)
    queryLimit = f"""select loanId from loans L
    where timestamp between UNIX_TIMESTAMP('2024-04-01') and UNIX_TIMESTAMP('2024-04-05')
    and L.loanIdx = 0
    and status not in ('standby')"""
    #if alreadyProcessedLoanIds:
     #   queryLimit += f""" and loanId not in ({alreadyProcessedLoanIds}); """

    print(queryLimit)
    cursor = connection.cursor()
    cursor.execute(queryLimit)
    results = cursor.fetchall()
   # columns = [desc[0] for desc in cursor.description]
    all_ids_list_df = pd.DataFrame(results)
    for i in range(0, len(all_ids_list_df), block_size):
        print(i)
        block_of_ids = all_ids_list_df[i:i + block_size]

        id_list = ','.join(map(str, block_of_ids[0].tolist()))
        if id_list:
            extract_and_save2(id_list, i, extract_query, connection)
        
    cursor.close()

finally:
    # Cerrar la conexión a la base de datos al finalizar
    connection.close()


ModuleNotFoundError: No module named 'pyarrow'

In [3]:

def getAlreadyProcessedLoanIds2():
    # Directorio que contiene los archivos CSV
    directorio = '/Users/sesaicornejorodriguez/baubap/demos/extractions/dcb/'

    # Lista para almacenar los DataFrames de los archivos CSV
    ids_unicos = set()

    # Enumerar todos los archivos en el directorio
    for archivo in os.listdir(directorio):
        if archivo.endswith('.csv'):
            # Construir la ruta completa del archivo
            print(archivo)
            ruta_archivo = os.path.join(directorio, archivo)
            
            # Leer solo la columna 'loanId' del archivo CSV y agregar los IDs únicos al conjunto
            df = pd.read_parquet(ruta_archivo, usecols=['loanId'])
            ids_unicos.update(df['loanId'])

    ids_concatenados = ','.join(str(id) for id in ids_unicos)

    return ids_concatenados 

print(getAlreadyProcessedLoanIds2())

NameError: name 'os' is not defined

In [26]:
directorio = '/Users/sesaicornejorodriguez/baubap/demos/extractions/dcb/febrero_2024_2_30000_data.csv'
import csv

try:

    file = pd.read_csv(directorio, sep=',', encoding="utf-8" , quoting=csv.QUOTE_NONE )

except pd.errors.ParserError as e:
    print("Error al leer el archivo CSV:", e)



Error al leer el archivo CSV: Error tokenizing data. C error: Expected 4 fields in line 3, saw 6

